# Unsupervised learning of multiple parameters

Before running this notebook: ensure that the steps outlined in sv_setup_instructions.txt are complete



In [ ]:
import torch
import pydpf
import sv_model
import pathlib
from sv_training_loop import train
import numpy as np
import pandas as pd
import os

## Experiment Settings

In [ ]:
batch_size_test = 128
batch_size_train = 32
true_alpha = 0.91
true_beta = 0.5
true_sigma = 1.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")   
temp_data_path = pathlib.Path('.').parent.absolute().joinpath('data/temp.csv')
experiment_cuda_rng = torch.Generator(device).manual_seed(0)
experiment_cpu_rng = torch.Generator().manual_seed(0)
experiments = ['DPF', 'Soft', 'Stop-Gradient', 'Marginal Stop-Gradient', 'Optimal Transport', 'Kernel']
n_repeats = 10
training_epochs = 20
result_path = pathlib.Path('.').parent.absolute().joinpath('results/multiple_parameter_results.csv')
if not result_path.exists():
    print("Results file does not exist. Make sure you have run the set-up script.")

In [ ]:
def get_SSM():
    alpha = torch.nn.Parameter(torch.rand((1,1), device=device, generator=experiment_cuda_rng), requires_grad=True)
    sigma = torch.nn.Parameter(torch.rand((1,1), device=device, generator=experiment_cuda_rng)*5, requires_grad=True)
    beta = torch.nn.Parameter(torch.rand((1,), device=device, generator=experiment_cuda_rng)*2, requires_grad=True)
    return sv_model.make_SSM(sigma, alpha, beta, device, experiment_cuda_rng), alpha, beta, sigma

In [ ]:
def get_DPF(SSM, DPF_type):
    if DPF_type == 'DPF':
        return pydpf.DPF(SSM=SSM, resampling_generator=experiment_cuda_rng)
    if DPF_type == 'Soft':
        return pydpf.SoftDPF(SSM=SSM, resampling_generator=experiment_cuda_rng)
    if DPF_type == 'Stop-Gradient':
        return pydpf.StopGradientDPF(SSM=SSM, resampling_generator=experiment_cuda_rng)
    if DPF_type == 'Marginal Stop-Gradient':
        return pydpf.MarginalStopGradientDPF(SSM=SSM, resampling_generator=experiment_cuda_rng)
    if DPF_type == 'Optimal Transport':
        return pydpf.OptimalTransportDPF(SSM=SSM, regularisation=0.5, transport_gradient_clip=1.)
    if DPF_type == 'Kernel':
        kernel = pydpf.StandardGaussian(1, experiment_cuda_rng, False, True)
        kernel_mixture = pydpf.KernelMixture(kernel, generator=experiment_cuda_rng)
        return pydpf.KernelDPF(SSM=SSM, kernel=kernel_mixture)
    raise ValueError('DPF_type should be one of the allowed options')

In [ ]:
for experiment in experiments:
    ELBOs = np.empty(n_repeats)
    alphas = np.empty(n_repeats)
    betas = np.empty(n_repeats)
    sigmas = np.empty(n_repeats)
    for n in range(n_repeats):
        experiment_cuda_rng = torch.Generator(device).manual_seed(n*10)
        generation_rng = torch.Generator(device).manual_seed(n*10)
        experiment_cpu_rng = torch.Generator().manual_seed(n*10)
        true_SSM = sv_model.make_SSM(torch.tensor([[true_sigma]], device=device), torch.tensor([[true_alpha]], device=device), torch.tensor([true_beta], device=device), device, generation_rng)
        pydpf.simulate_and_save(temp_data_path, SSM=true_SSM, time_extent=1000, n_trajectories=500, batch_size=batch_size_test, device=device, bypass_ask=True)
        SSM, alpha, beta, sigma = get_SSM()
        dpf = get_DPF(SSM, experiment)
        if experiment == 'Kernel':
            opt = torch.optim.SGD([{'params':[alpha], 'lr':0.05}, {'params':[beta], 'lr':0.1}, {'params':[sigma], 'lr':0.25}, {'params':dpf.resampler.mixture.parameters(), 'lr':0.1}], lr=0.2, momentum=0.9, nesterov=True)
        else:
            opt = torch.optim.SGD([{'params':[alpha], 'lr':0.05}, {'params':[beta], 'lr':0.1}, {'params':[sigma], 'lr':0.25}], lr=0.2, momentum=0.9, nesterov=True)
        opt_schedule = torch.optim.lr_scheduler.ExponentialLR(opt, 0.95)
        dataset = pydpf.StateSpaceDataset(temp_data_path, state_prefix='state', device=device)
        _, ELBO = train(dpf, opt, dataset, training_epochs, (100, 100, 100), (batch_size_train, batch_size_test, batch_size_train), (0.5, 0.25, 0.25), 1., experiment_cpu_rng, target='ELBO', time_extent=100, lr_scheduler=opt_schedule)
        ELBOs[n] = ELBO
        alphas[n] = alpha
        betas[n] = beta
        sigmas[n] = sigma
        os.remove(temp_data_path)
    results = pd.read_csv(result_path, index_col=0)
    row = np.array([np.mean(ELBOs), np.mean(np.abs(alphas - 0.91)), np.mean(np.abs(betas - 0.5)), np.mean(np.abs(sigmas - 1.))])
    results.loc[experiment] = row
    results.to_csv(result_path)